In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scipy import stats

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [27]:
# Cargar el dataset limpio del paso EDA
df = pd.read_csv('https://raw.githubusercontent.com/aliciacaminero/Gemali_Cars/refs/heads/main/Predictor%20de%20precio%20de%20venta/df_EDA_predicprecio.csv')

df

,make,model,version,fuel,year,kms,power,shift,price
0,Opel,Crossland,1.2 GAS 110 GS Line 5p S/S,Gasolina,2022,5.0,110.0,manual,22900
1,Opel,Crossland,1.2 81kW (110CV) GS Line,Gasolina,2022,24847.0,110.0,manual,19990
2,Opel,Crossland,1.5D 88kW (120CV) Business Elegance Auto,Diésel,2021,41356.0,120.0,automatic,18590
3,Opel,Crossland,GS-Line 1.2 GAS MT6 S/S 110cv,Gasolina,2022,11.0,110.0,manual,22700
4,Opel,Crossland,1.2 GS LINE 110 CV 5P,Gasolina,2021,51390.0,110.0,manual,18200
...,...,...,...,...,...,...,...,...,...
12448,Citroen,C4 Picasso,BlueHDi 88KW (120CV) Feel,Diésel,2017,71330.0,120.0,manual,18200
12449,Citroen,C4 Picasso,BlueHDi 150cv Exclusive,Diésel,2014,109350.0,150.0,manual,11690
12450,Citroen,C4 Picasso,PureTech SS 6v Live,Gasolina,2017,108697.0,130.0,manual,13199
12451,Citroen,C4 Picasso,Seduction,Diésel,2013,173035.0,111.0,automatic,9290


In [28]:
'''le_fuel = LabelEncoder()
df['fuel'] = le_fuel.fit_transform(df['fuel'])
# 2 = Gasolina, 0 = Diesel, 1 = Electrico

le_shift = LabelEncoder
df['shift'] = le_fuel.fit_transform(df['shift'])

# Crear el objeto LabelEncoder
le_make = LabelEncoder()

# Codificar la columna 'make'
df['make'] = le_make.fit_transform(df['make'])

# Crear un LabelEncoder para la columna 'model'
le_model = LabelEncoder()

# Aplicamos el LabelEncoder en la columna 'model'
df['model'] = le_model.fit_transform(df['model'])'''

"le_fuel = LabelEncoder()\ndf['fuel'] = le_fuel.fit_transform(df['fuel'])\n# 2 = Gasolina, 0 = Diesel, 1 = Electrico\n\nle_shift = LabelEncoder\ndf['shift'] = le_fuel.fit_transform(df['shift'])\n\n# Crear el objeto LabelEncoder\nle_make = LabelEncoder()\n\n# Codificar la columna 'make'\ndf['make'] = le_make.fit_transform(df['make'])\n\n# Crear un LabelEncoder para la columna 'model'\nle_model = LabelEncoder()\n\n# Aplicamos el LabelEncoder en la columna 'model'\ndf['model'] = le_model.fit_transform(df['model'])"

In [29]:
# Definimos cuáles son las características numéricas, incluyendo las nuevas
numeric_features = ['year', 'kms', 'power']
categorical_features = ['fuel', 'shift', 'make', 'model']

# Crear un diccionario para almacenar los codificadores
label_encoders = {}

# Lista de columnas a codificar -> categorical_features

# Codificar cada columna y guardar el codificador
for column in categorical_features:
    le = LabelEncoder()  # Crear un nuevo LabelEncoder para cada columna
    df[column] = le.fit_transform(df[column])  # Aplicamos la transformación a la columna
    label_encoders[column] = le  # Guardamos el codificador en el diccionario


In [30]:
# Características originales
features = ['make', 'model', 'fuel', 'year', 'kms', 'shift', 'power']
target = 'price'

# Separamos en variables originales
X = df[features].copy()
y = df[target]

In [31]:


# Detectamos outliers mediante Z-score
# Calculamos los Z-scores para características numéricas
#z_scores = stats.zscore(X[numeric_features])
##abs_z_scores = np.abs(z_scores)

# Consideramos outliers valores con Z-score > 3 (muy alejados de la media)
#filtered_entries = (abs_z_scores < 3).all(axis=1)

# Filtramos el dataset para eliminar los outliers
#X_filtered = X[filtered_entries]
#y_filtered = y[filtered_entries]

# Comprobamos cuántos registros se eliminaron
#print(f"Registros originales: {len(X)}")
#print(f"Registros después de eliminar outliers: {len(X_filtered)}")
#print(f"Registros eliminados: {len(X) - len(X_filtered)} ({(len(X) - len(X_filtered))/len(X)*100:.2f}%)")

In [32]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Verificamos las dimensiones de los conjuntos
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (9962, 7)
X_test shape: (2491, 7)
y_train shape: (9962,)
y_test shape: (2491,)


In [33]:
# Definimos el preprocesador para las características numéricas y categóricas
preprocessor = ColumnTransformer(
    transformers=[
        # Estandarizamos variables numéricas (media 0, desviación 1)
        ('num', StandardScaler(), numeric_features),
        # Codificamos variables categóricas con one-hot encoding
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [34]:
# Creamos el modelo de Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenamos el modelo con los datos de entrenamiento
rf_model.fit(X_train , y_train)

# Hacemos predicciones sobre el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Suponiendo que tienes y_test (valores reales) y y_pred (predicciones del modelo)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")

from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

Mean Absolute Error (MAE): 3276.7482710715954
Mean Squared Error (MSE): 44112628.140074894
R²: 0.9274120228278255


In [35]:
# Mostrar las columnas utilizadas para el modelo
columnas_utilizadas = X_train.columns.tolist()
print(columnas_utilizadas)

# target es price

['make', 'model', 'fuel', 'year', 'kms', 'shift', 'power']


In [36]:
import joblib

In [37]:
# Guardar el modelo y los label encoders usando joblib
joblib.dump({'model': rf_model, 'label_encoders': label_encoders}, 'model_with_encoders.joblib', compress=('zlib', 3))

print("Modelo y LabelEncoders guardados exitosamente.")

Modelo y LabelEncoders guardados exitosamente.
